In [141]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt

# Cargamos los datos
datos = pd.read_csv("water_potability.csv")
means = datos.mean(axis=0, skipna=True)

for e in datos:
    datos[e] = datos[e].fillna(means[e])
    
Y = datos.iloc[:,-1]
Y = np.array(Y)
datos = datos.iloc[:,:-1]
datos
#Hasta aqui datos cargados y sustituimos los nan por la media

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
0,7.080795,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135
1,3.716080,129.422921,18630.057858,6.635246,333.775777,592.885359,15.180013,56.329076,4.500656
2,8.099124,224.236259,19909.541732,9.275884,333.775777,418.606213,16.868637,66.420093,3.055934
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075
...,...,...,...,...,...,...,...,...,...
3271,4.668102,193.681735,47580.991603,7.166639,359.948574,526.424171,13.894419,66.687695,4.435821
3272,7.808856,193.553212,17329.802160,8.061362,333.775777,392.449580,19.903225,66.396293,2.798243
3273,9.419510,175.762646,33155.578218,7.350233,333.775777,432.044783,11.039070,69.845400,3.298875
3274,5.126763,230.603758,11983.869376,6.303357,333.775777,402.883113,11.168946,77.488213,4.708658


In [129]:
def normalize1(df):
    result = df.copy()
    for feature_name in df.columns:
        mean = df[feature_name].mean()
        std = df[feature_name].std()
        result[feature_name] = (df[feature_name] - mean) / std
    return result

def normalize2(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

#definimos funciones de normalizacion
X = np.asarray(datos)
X_norm1 = normalize1(datos)
X_norm2 = normalize2(datos)
X1 = np.asarray(X_norm1)
X2 = np.asarray(X_norm2)

filas = np.shape(X1)[0]
columnas = np.shape(X1)[1]

In [130]:
from sklearn.model_selection import train_test_split
train_size = 0.7
validation_size = 0.1
test_size = 0.2

validation = validation_size / (test_size + validation_size)
x_train, x_test, y_train, y_test = train_test_split(X1, Y, train_size=train_size)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, train_size=validation)

x_train = np.hstack([np.ones([np.shape(x_train)[0],1]),x_train])
x_val = np.hstack([np.ones([np.shape(x_val)[0],1]),x_val])
x_test = np.hstack([np.ones([np.shape(x_test)[0],1]),x_test])

In [12]:
#datos = datos puros en pandas
#Y = array de resultados
#X1 = matriz normalizada de los datos en numpy
#X1 = matriz con segunda normalizacion
#x_train = conjunto de entrenamiento con columna de 1
#x_val = conjunto de validacion con columna de 1
#X_test = conjunto de test con columna de 1

# Regresion lineal

In [155]:
def hip(X,Y):
    return np.dot(X,Y)
#Funcion de coste
def cost(X,Y, filas):
    #hip = np.dot(X, theta)
    #aux = (hip - Y) **2
    #return aux.sum() / (2 * filas)
    (1/2) * filas * sum((hip(X)-Y)**2)

    
# Metodo de descenso de gradiente vectorizado
def gradient(X, Y, theta, alpha, filas):
    th = theta
    for i in range(columnas):
        aux = ((np.dot(X, theta) - Y) * X[:, i])
        th[i] -= ((alpha/filas) * aux.sum())
    return th#, fun_coste(X, Y, th, filas)
    theta[0] = (theta - (alpha/filas)) * sum(hip(X) - Y)
    el resto = theta[0]*X

# Ecuacion normal
def ecuacion_normal(X, Y, filas):
    X= np.hstack([np.ones([filas,1]),X])
    th = np.linalg.inv(np.transpose(X).dot(X)).dot(np.transpose(X)).dot(Y)
    return th

def porcentaje(X, Y, thetas, filas):
    ok = 0
    i = 0
    for h in np.dot(X, thetas):
        if h >= 0.5:
            if Y[i] == 1.0:
                ok +=1
        else:
            if Y[i] == 0.0:
                ok +=1
        i +=1
    return (ok / filas)

Vamos a hacer la regresion lineal con la ecuacion normal

In [156]:
thetas = ecuacion_normal(datos, Y, filas)
XO = np.hstack([np.ones([filas,1]),X])
print(porcentaje(XO,Y, thetas,filas ))

0.6101953601953602


In [153]:
thg = np.zeros(columnas + 1)
alpha = 0.1

for i in range(3000):
    thg, cost = gradiente(x_train, y_train, thg, alpha,filas)
    
print(thg)

[ 0.39150807  0.00209323 -0.00538415  0.01479422  0.02317298 -0.01323043
 -0.0106171  -0.00651375  0.00255757  0.        ]


In [160]:
theta = [0] * (columnas)
result = opt.fmin_tnc(func=cost, x0=theta, fprime=gradient, args=(x_train, y_train, filas))
theta_opt = result[0]

ValueError: shapes (9,) and (2293,) not aligned: 9 (dim 0) != 2293 (dim 0)

In [154]:
print(porcentaje(XO,Y, thg,filas))

0.3904151404151404
